In [78]:
import os
import io

from google.cloud import vision
from google.cloud.vision_v1 import types
import http.client

In [79]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'tcnjhack-351c09114ab7.json'

In [80]:
client = vision.ImageAnnotatorClient()

In [81]:
import cv2

cam = cv2.VideoCapture(0)

cv2.namedWindow("test")

img_counter = 0

while True:
    ret, frame = cam.read()
    if not ret:
        print("failed to grab frame")
        break
    cv2.imshow("test", frame)

    #k = cv2.waitKey(1)
    #if k%256 == 27:
        # ESC pressed
        #print("Escape hit, closing...")
        #break
    #elif k%256 == 32:
        # SPACE pressed
        
    cv2.waitKey(2000)
    img_name = "opencv_frame_{}.png".format(img_counter)
    cv2.imwrite(img_name, frame)
    print("{} written!".format(img_name))
    img_counter += 1
    if img_counter > 14:
        print("15 iterations completed, closing...")
        break

cam.release()

cv2.destroyAllWindows()

opencv_frame_0.png written!
opencv_frame_1.png written!
opencv_frame_2.png written!
opencv_frame_3.png written!
opencv_frame_4.png written!
opencv_frame_5.png written!
opencv_frame_6.png written!
opencv_frame_7.png written!
opencv_frame_8.png written!
opencv_frame_9.png written!
opencv_frame_10.png written!
opencv_frame_11.png written!
opencv_frame_12.png written!
opencv_frame_13.png written!
opencv_frame_14.png written!
15 iterations completed, closing...


In [97]:
FILE_NAME = 'opencv_frame_{}.png'.format(img_counter - 1)
FOLDER_PATH = r'C:\Users\arnav\googleapi_proj'

In [98]:
with io.open(os.path.join(FOLDER_PATH, FILE_NAME), 'rb') as image_file:
        content = image_file.read()

In [99]:
image = vision.Image(content=content)

In [110]:
emotionArr = []
sumJOY = 0 
sumSOR = 0
sumSURP = 0
sumANG = 0
unknownArr = [0, 0, 0, 0]
tot_img = 15

for x in range(0, 15):
    FILE_NAME = 'opencv_frame_{}.png'.format(x)
    with io.open(os.path.join(FOLDER_PATH, FILE_NAME), 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)
    response_face = client.face_detection(image=image)

    face_data = []

    for face_detection in response_face.face_annotations: 
        d = { 
            'confidence': face_detection.detection_confidence, 
            'joy': face_detection.joy_likelihood, 
            'sorrow': face_detection.sorrow_likelihood, 
            'surprise': face_detection.surprise_likelihood, 
            'anger': face_detection.anger_likelihood 
            }
    print(d)
    list2 = list(d.values())
    print(list2)
    list3 = []
        #print("111")
        
    #print(str(list2[0]))
    number = 0
    count = 0
    for i in list2:
        if ".VERY_LIKELY" in str((list2[count])):
            number = number + 5
        elif ".POSSIBLE" in str((list2[count])):
            number = number + 3
        elif ".VERY_UNLIKELY" in str((list2[count])):
            number = number + 1
        elif ".UNLIKELY" in str((list2[count])):
            number = number + 2
        elif ".LIKELY" in str((list2[count])):
            number = number + 4
        elif ".UNKNOWN" in str ((list2[count])):
            unknown[count] += 1
            
        
        list3.append(number)
        
        count += 1
        number = 0
    #print(list3)
        #print("count: ")
        #print(count)
    #print(number)
        

    list3.remove(0)
    sumJOY += list3[0]
    sumSOR += list3[1]
    sumSURP += list3[2]
    sumANG += list3[3]
    
    print(list3)
    print(sumJOY)
    print(sumSOR) 
    print(sumSURP)
    print(sumANG)
    print(emotionArr)


emotionArr.append(sumJOY)
emotionArr.append(sumSOR)
emotionArr.append(sumSURP)
emotionArr.append(sumANG)
print(emotionArr)


    

{'confidence': 0.7023523449897766, 'joy': <Likelihood.VERY_UNLIKELY: 1>, 'sorrow': <Likelihood.VERY_UNLIKELY: 1>, 'surprise': <Likelihood.VERY_UNLIKELY: 1>, 'anger': <Likelihood.VERY_UNLIKELY: 1>}
[0.7023523449897766, <Likelihood.VERY_UNLIKELY: 1>, <Likelihood.VERY_UNLIKELY: 1>, <Likelihood.VERY_UNLIKELY: 1>, <Likelihood.VERY_UNLIKELY: 1>]
[1, 1, 1, 1]
1
1
1
1
[]
{'confidence': 0.8308629989624023, 'joy': <Likelihood.VERY_LIKELY: 5>, 'sorrow': <Likelihood.VERY_UNLIKELY: 1>, 'surprise': <Likelihood.VERY_UNLIKELY: 1>, 'anger': <Likelihood.VERY_UNLIKELY: 1>}
[0.8308629989624023, <Likelihood.VERY_LIKELY: 5>, <Likelihood.VERY_UNLIKELY: 1>, <Likelihood.VERY_UNLIKELY: 1>, <Likelihood.VERY_UNLIKELY: 1>]
[5, 1, 1, 1]
6
2
2
2
[]
{'confidence': 0.8968009352684021, 'joy': <Likelihood.VERY_LIKELY: 5>, 'sorrow': <Likelihood.VERY_UNLIKELY: 1>, 'surprise': <Likelihood.VERY_UNLIKELY: 1>, 'anger': <Likelihood.VERY_UNLIKELY: 1>}
[0.8968009352684021, <Likelihood.VERY_LIKELY: 5>, <Likelihood.VERY_UNLIKELY: 

In [111]:
joy_avg = emotionArr[0] / (tot_img - unknownArr[0])
print(joy_avg)
sor_avg = emotionArr[1] / (tot_img - unknownArr[1])
print(sor_avg)
surp_avg = emotionArr[2] / (tot_img - unknownArr[2])
print(surp_avg)
ang_avg = emotionArr[3] / (tot_img - unknownArr[3])
print(ang_avg)

2.3333333333333335
1.0
1.2
1.0


In [116]:
sentiment_score = (((3* joy_avg) + (-2 * sor_avg) + (surp_avg) + (-2* ang_avg)) / 4)
#sentiment_score = (((joy_avg) + (sor_avg) + (surp_avg) + (-ang_avg)) / 4)
print(sentiment_score)

1.05


In [124]:
if(x<=-14 + x>=-16):
    print("Very strongly do not recommend")
elif(x<=-10 + x>-14):
    print("Strongly do not recommend")
elif(x<=-6 + x>-10):
    print("Do not recommend")
elif(x<=-2 + x>-6):
    print("Might not recommend")
elif(-2<x + 2>x):
    print("User does not show enough emotion to determine recommendation")
elif(x<=2 + x>6):
    print("Might recommend")
elif(x<=6 + x>10):
    print("Do recommend")
elif(x<=10 + x>14):
    print("Strongly recommend")
elif(x<=14 + x>=16):
    print("Very strongly recommend")

User does not show enough emotion to determine recommendation
